# Imports

In [ ]:
import librosa

import numpy as np
import pandas as pd
import random

import torch
import torchmetrics
import torchaudio
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB
import os
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

### Config

In [ ]:
class Config:
    FILENAME = 'Wav2Vec'
    
    SR = 32000
    N_MFCC = 40
    TARGET_LENGTH = 1024
    
    # Dataset
    ROOT_DIR = 'C:/HongBeomsun/Dataset_SSD/FakeVoice'
    
    # Training
    N_CLASSES = 2
    BATCH_SIZE = 64
    N_EPOCHS = 70
    LEARNING_RATE = 3e-4
    
    # Others
    SEED = 42
    
CONFIG = Config()

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
seed_everything(CONFIG.SEED)

### Data

In [ ]:
df = pd.read_csv(os.path.join(CONFIG.ROOT_DIR,'train.csv'))

In [ ]:
print(len(df))
df.head()

In [ ]:
df['label'].value_counts()

### Data Argumentation

### Train test split

In [ ]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CONFIG.SEED, stratify=df['label'])

In [ ]:
train['label'].value_counts()
val['label'].value_counts()

### Feature Extraction

In [ ]:
def noise(data):
    noise_amp = 0.01*np.random.uniform()*np.amax(data)
    data = data + noise_amp * np.random.normal(size=data.shape[0])
    return data

def stretch(data, rate=0.8):
    stretch_data = librosa.effects.time_stretch(data, rate=rate)
    return stretch_data

def pitch(data, sampling_rate, pitch_factor=0.7):
    pitch_data = librosa.effects.pitch_shift(y=data, sr=sampling_rate, n_steps=pitch_factor)
    return pitch_data

In [ ]:
def load_audio(filepath, sr):
    waveform, sr = torchaudio.load(filepath)
    return waveform, sr

def extract_features(waveform, sr, target_length):
    mel_spec = MelSpectrogram(sr)(waveform)
    db_spec = AmplitudeToDB()(mel_spec)
    
    if db_spec.shape[-1] < target_length:
        pad_amount = target_length - db_spec.shape[-1]
        db_spec = F.pad(db_spec, (0, pad_amount))
        
    else:
        db_spec = db_spec[:, :, :target_length]
        
    return db_spec

def normalize_volume(y, target_dB=-20):
    rms = np.sqrt(np.mean(y**2))
    loudness = 20 * np.log10(rms)
    loudness_change_dB = target_dB - loudness
    y_normalized = y * (10 ** (loudness_change_dB / 20))
    return y_normalized

def augment_data(y, sr):
    augmented_data = []
    augmented_data.append(noise(y))
    augmented_data.append(stretch(y))
    augmented_data.append(pitch(y, sr))
    return augmented_data

def create_void_data(data, sr):
    void_data = np.zeros_like(data)
    void_data = noise(void_data)
    return void_data

def create_duo_data(data1, data2, sr):
    if len(data1) > len(data2):
        data2 = np.pad(data2, (0, len(data1)-len(data2)), 'constant')
    else:
        data1 = np.pad(data1, (0, len(data2)-len(data1)), 'constant')
        
    duo_data = data1 + data2
    max_val = np.max(np.abs(duo_data))
    if max_val > 1:
        duo_data = duo_data / max_val
    
    return duo_data

def mix_two_random_data(df, sr):
    idx1, idx2 = random.sample(range(len(df)), 2)
    y1, _ = load_audio(os.path.join(CONFIG.ROOT_DIR, df.iloc[idx1]['path']), sr)
    y2, _ = load_audio(os.path.join(CONFIG.ROOT_DIR, df.iloc[idx2]['path']), sr)
    y_duo = create_duo_data(y1, y2, sr)
    label_y1 = df.iloc[idx1]['label']
    label_y2 = df.iloc[idx2]['label']
    
    label_vector = np.zeros(CONFIG.N_CLASSES, dtype=float)
    label_vector[0 if label_y1 == 'fake' else 1] = 1
    label_vector[0 if label_y2 == 'fake' else 1] = 1
    
    return y_duo, label_vector

In [ ]:
def get_features(df, train_mode=True, augment=False):
    features = []
    labels = []
    total = len(df)
    
    for i, (index, row) in enumerate(tqdm(df.iterrows(), total=total), 1):
        y, sr = load_audio(os.path.join(CONFIG.ROOT_DIR, row['path']), CONFIG.SR)
        
        if train_mode:
            label = row['label']
            label_vector = np.zeros(CONFIG.N_CLASSES, dtype=float)
            label_vector[0 if label == 'fake' else 1] = 1
            labels.append(label_vector)
        
        features.append(extract_features(y, sr, target_length=CONFIG.TARGET_LENGTH))
        
    if train_mode:
        return features, labels
    return features

In [ ]:
"""
    if augment:
        augmented_features = []
        augmented_labels = []
        
        num_mixed_samples = int(total * 0.1)
        for _ in range(num_mixed_samples):
            y_duo, y_duo_label = mix_two_random_data(df, CONFIG.SR)
            augmented_features.append(extract_features(y_duo, CONFIG.SR))
            augmented_labels.append(y_duo_label)
        
        num_augmented_samples = int(total * 0.2)
        original_features = list(features)
        original_labels = list(labels)
        for idx in range(num_augmented_samples):
            augmented_data = augment_data(original_features[idx], CONFIG.SR)
            for aug_y in augmented_data:
                augmented_features.append(extract_features(aug_y, CONFIG.SR))
                augmented_labels.append(original_labels[idx])
        
        features.extend(augmented_features)
        labels.extend(augmented_labels)"""

In [ ]:
train_features, train_labels = get_features(train, train_mode=True, augment=True)
val_features, val_labels = get_features(val, train_mode=True, augment=False)

In [ ]:
def check_shape(features):
    shapes = [feature.shape for feature in features]
    unique_shape = set(shapes)
    return unique_shape

print(check_shape(train_features))

In [ ]:
def save_np():
    os.makedirs(os.path.join(CONFIG.ROOT_DIR, 'npy', CONFIG.FILENAME, 'npy'), exist_ok=True)
    np.save(os.path.join(CONFIG.ROOT_DIR, 'npy', CONFIG.FILENAME, 'train_features.npy'), train_features)
    np.save(os.path.join(CONFIG.ROOT_DIR, 'npy', CONFIG.FILENAME, 'train_labels.npy'), train_labels)
    np.save(os.path.join(CONFIG.ROOT_DIR, 'npy', CONFIG.FILENAME, 'val_features.npy'), val_features)
    np.save(os.path.join(CONFIG.ROOT_DIR, 'npy', CONFIG.FILENAME, 'val_labels.npy'), val_labels)

In [ ]:
def load_np():
    train_features = np.load(os.path.join(CONFIG.ROOT_DIR, CONFIG.FILENAME, 'train_features.npy'))
    train_labels = np.load(os.path.join(CONFIG.ROOT_DIR, CONFIG.FILENAME, 'train_labels.npy'))
    val_features = np.load(os.path.join(CONFIG.ROOT_DIR, CONFIG.FILENAME, 'val_features.npy'))
    val_labels = np.load(os.path.join(CONFIG.ROOT_DIR, CONFIG.FILENAME, 'val_labels.npy'))
    
    return train_features, train_labels, val_features, val_labels

In [ ]:
def save_np_in_chunks(data, dataname, chunk_size=10000):
    os.makedirs(os.path.join(CONFIG.ROOT_DIR,'npy', CONFIG.FILENAME), exist_ok=True)
    
    total_chunks = len(data) // chunk_size + (1 if len(data) % chunk_size != 0 else 0)
    for i in range(total_chunks):
        chunk_data = data[i*chunk_size:(i+1)*chunk_size]
        np.savez_compressed(os.path.join(CONFIG.ROOT_DIR, 'npy', CONFIG.FILENAME, f"dataname_part{i+1}.npz"), chunk_data)

def load_np_in_chunks(total_chunks):
    data = []
    for i in range(total_chunks):
        chunk_data = np.load(os.path.join(CONFIG.ROOT_DIR, 'npy', CONFIG.FILENAME, f"dataname_part{i+1}.npz"))['arr_0']
        data.append(chunk_data)
    return np.concatenate(data, axis=0)

In [ ]:
save_np_in_chunks(train_features, 'train_features' , chunk_size=1000)
save_np_in_chunks(train_labels, 'train_labels', chunk_size=1000)

train_features = load_np_in_chunks(total_chunks=50)
train_labels = load_np_in_chunks(total_chunks=50)

In [ ]:
train_features = np.array(train_features)
train_labels = np.array(train_labels)
val_features = np.array(val_features)
val_labels = np.array(val_labels)

In [ ]:
print(train_features.shape, len(train_labels))
print(val_features.shape, len(val_labels))

### Define Model

In [ ]:
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Tokenizer

model_name = 'facebook/wav2vec2-base-960h'
model = Wav2Vec2ForSequenceClassification.from_pretrained(model_name, num_labels=CONFIG.N_CLASSES)
tokenizer = Wav2Vec2Tokenizer.from_pretrained(model_name)

In [ ]:
def tokenize_audio(audio):
    input_values = tokenizer(audio, return_tensors="pt", padding="longest").input_values
    return input_values

### DataLoader

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, filepaths, labels):
        self.filepaths = filepaths
        self.labels = labels
    
    def __len__(self):
        return len(self.filepaths)
    
    def __getitem__(self,idx):
        filepath = self.filepaths[idx]
        label = self.labels[idx]
        waveform, sr = load_audio(filepath)
        input_values = tokenize_audio(waveform)
        return input_values, label

In [ ]:
train_dataset = CustomDataset(train_features, train_labels)
val_dataset = CustomDataset(val_features, val_labels)

In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=False
)

### MLFlow

In [ ]:
import mlflow
import mlflow.pytorch

mlflow.set_experiment('FakeVoice')

def mlflow_run_decorator(run_name=None):
    def decorator(func):
        def wrapper(*args, **kwargs):
            mlflow.start_run(run_name=run_name)
            try:
                result = func(*args, **kwargs)
                mlflow.set_tag("Status", "SUCCEESS")
            except Exception as e:
                mlflow.log_param("Exception", e)
                mlflow.set_tag("Status", "FAIL")
                raise e
            finally:
                mlflow.end_run()
            return result
        return wrapper
    return decorator

### Train & Validation

In [ ]:
from sklearn.metrics import roc_auc_score
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

In [ ]:
@mlflow_run_decorator(run_name=os.path.basename(__file__))
def training(model, scheduler, optimizer, train_loader, val_loader, device):
    mlflow.log_params(vars(CONFIG))
    
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_val_score = 0
    best_model = None
    
    for epoch in range(1, CONFIG.N_EPOCHS+1):
        model.train()
        train_loss = []
        for features, labels in tqdm(iter(train_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)
            
            optimizer.zero_grad()
            
            output = model(features)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val AUC : [{_val_score:.5f}] LEARNING RATE : [{optimizer.param_groups[0]["lr"]:.5f}]')
    
        mlflow.log_metrics({'train_loss': _train_loss, 'val_loss': _val_loss, 'val_auc': _val_score}, step=epoch)
        scheduler.step(_val_loss)
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
    
    return best_model

In [ ]:
def multiLabel_AUC(y_true, y_scores):
    auc_scores = []
    for i in range(y_true.shape[1]):
        auc = roc_auc_score(y_true[:, i], y_scores[:, i])
        auc_scores.append(auc)
    mean_auc_score = np.mean(auc_scores)
    return mean_auc_score

In [ ]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss, all_labels, all_probs = [], [], []
    
    with torch.no_grad():
        for features, labels in tqdm(iter(val_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)
            
            probs = model(features)
            
            loss = criterion(probs, labels)

            val_loss.append(loss.item())

            all_labels.append(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())
        
        _val_loss = np.mean(val_loss)

        all_labels = np.concatenate(all_labels, axis=0)
        all_probs = np.concatenate(all_probs, axis=0)
        
        # Calculate AUC score
        auc_score = multiLabel_AUC(all_labels, all_probs)
    
    return _val_loss, auc_score

### Run

In [ ]:
model = MLP()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CONFIG.LEARNING_RATE)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

infer_model = training(model, scheduler, optimizer, train_loader, val_loader, device)

### Inference

In [ ]:
test = pd.read_csv(os.path.join(CONFIG.ROOT_DIR, 'test.csv'))
test_mfcc = get_features(test, train_mode=False, augment=False)

In [ ]:
test_dataset = CustomDataset(test_mfcc, None)
test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=False
)

In [ ]:
# np.save(os.path.join(CONFIG.ROOT_DIR, 'npy/test_VariousFeatures_1000.npy'), test_mfcc)
# test_mfcc = np.load(os.path.join(CONFIG.ROOT_DIR, 'npy/test_VariousFeatures_1000.npy'))

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for features in tqdm(iter(test_loader)):
            features = features.float().to(device)
            
            probs = model(features)

            probs  = probs.cpu().detach().numpy()
            predictions += probs.tolist()
    return predictions

In [ ]:
preds = inference(infer_model, test_loader, device)

### Submission

In [ ]:
submit = pd.read_csv(os.path.join(CONFIG.ROOT_DIR,'./sample_submission.csv'))
submit.iloc[:, 1:] = preds
submit.head()

In [ ]:
submit.to_csv(f'./output/submit_MLP_Augment.csv', index=False)

### AfterTest

In [ ]:
print(model(torch.tensor(train_features).float().to(device)).cpu().detach().numpy()[:10])
print(train_labels[:10])

In [ ]:
np.where((train_labels[:, 0] == 1) & (train_labels[:, 1] == 1))[0]